In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, accuracy_score, roc_auc_score, r2_score
from sklearn.metrics import mean_squared_error
from tensorflow import keras

In [ ]:
names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO',
'B', 'LSTAT', 'MEDV']
df = pd.read_csv("../input/boston-house-prices/housing.csv", delim_whitespace=True, names=names)


**EDA**

In [ ]:
display(df)
plt.hist(df["MEDV"]);

**Feature Selection**

In [ ]:
plt.figure(figsize=(10,8))
cm = df.corr()
sns.set(font_scale=1)
sns.heatmap(cm,annot=True,cmap= plt.cm.Reds,fmt= ".0%")

INDUS, RM, TAX, PTRATIO and LSTAT has good corralation with target feature MEDV. But INDUS also has high correlation with LTSAT and TAX. So in this case i should not use INDUS. The best features are RM, TAX, PTRATIO and LSTAT.

In [ ]:
df = df[["RM", "TAX", "PTRATIO","LSTAT", "MEDV"]]
display(df)

**Normalization**

In [ ]:
df = (df - df.mean()) / df.std()

In [ ]:
def train_test_split(df,test_size,shuffle): # from scratch 
    if shuffle:
        df = df.reindex(np.random.permutation(df.index))
    total_sample_size= len(df)
    train_end = round(total_sample_size*(1-test_size))
    train = df[0:train_end]
    test = df[train_end:total_sample_size]
    return train,test

def cross_validation(df,fold,train_size,feature_count): # from scratch 
    dataset_size = len(df) 
     
    division_num = 1//(1-train_size)
    kfold = np.zeros((dataset_size//fold,feature_count), dtype=int)
    val_size = int(dataset_size*(1-train_size))
    train_size = int(dataset_size*train_size)
    val = np.zeros((val_size,feature_count),dtype=int)
    train = np.zeros((train_size,feature_count),dtype=int)
    
    for i in range(fold):
        kfold = np.append(kfold,df[i*(dataset_size//fold):((i+1)*dataset_size//fold)-1].to_numpy(),axis=0)
        # first column is zero
        val = np.append(val,kfold[int((i+1)*dataset_size//fold):int((i+1)*dataset_size//fold + val_size//fold )-1],axis=0)
        train = np.append(train,kfold[int((i+1)*dataset_size//fold + val_size//fold ):int((i+1)*dataset_size//fold + val_size//fold + train_size//fold)-1],axis=0)
    val = np.delete(val,np.linspace(0,val_size-1,val_size,dtype=int),0)
    train = np.delete(train,np.linspace(0,train_size-1,train_size,dtype=int),0)
    return train, val

train, test = train_test_split(df,test_size=0.16,shuffle=True)        
train, val = cross_validation(train,fold=5,train_size = 0.79,feature_count=5)

df_train = pd.DataFrame(train,columns=df.columns)
df_val = pd.DataFrame(val,columns=df.columns)
df_test = pd.DataFrame(test,columns=df.columns)
total_data_size = len(df)


print("Train ratio: %",len(train)/total_data_size*100)
print("Test ratio: %",len(test)/total_data_size*100)
print("Validation ratio: %",len(val)/total_data_size*100)
print("Train/Test ratio: %",len(train)/(len(train)+len(test))*100,"- %", len(test)/(len(train)+len(test))*100)
# requested train/test ratio was %80 - %20

x_train, y_train = df_train.drop(columns="MEDV").values.tolist(), df_train.MEDV.values.tolist()
x_val, y_val = df_val.drop(columns="MEDV").values.tolist(), df_val.MEDV.values.tolist()
x_test, y_test = df_test.drop(columns="MEDV").values.tolist(), df_test.MEDV.values.tolist()

In [ ]:
def r_squared(y_true,y_predicted):
    mse_model = np.square(np.subtract(y_true,y_pred)).mean() # variance of model
    mse_baseline = np.square(np.subtract(y_pred.mean(),y_pred)).mean() # variance of target variable
    r_squared = 1 - (mse_model/mse_baseline)
    return r_squared

In [ ]:
class SimpleNeuralNetwork:
    
    def __init__(self): 
        
        '''        
        Initial weights and biases assigned random values ranging '0' to '1'. 
        We have a total of 9 weights and 4 biases.
        6 weights are coming in the hidden layer, two for each neuron, hence 3 x 2 = 6.
        The rest of the weights are coming into the output layer.
        Same story for the biases. Each bias is attached to the neuron in the hidden layer
        and output layer.
        
        output layer has one neuron because its a regression problem
        '''
        # Weights
        self.w1, \
        self.w2, \
        self.w3, \
        self.w4, \
        self.w5, \
        self.w6, \
        self.w7, \
        self.w8, \
        self.w9, \
        self.w10, \
        self.w11, \
        self.w12, \
        self.w13, \
        self.w14, \
        self.w15, \
        self.w16, \
        self.w17, \
        self.w18, \
        self.w19, \
        self.w20 = np.random.rand(20)

        # Biases
        self.b_n1, \
        self.b_n2, \
        self.b_n3, \
        self.b_n4, \
        self.b_y_hat = np.random.rand(5) 

    # Activation function for the neurons
    # Each neuron IS an actually activation function itself
    # sigmoid is for forward propagation, sigmoid derivative is for back propagation
    def sigmoid(self, x): return 1 / (1 + np.e**-x)
    def sigmoid_der(self, x): return self.sigmoid(x) * (1 - self.sigmoid(x))

    
    '''
    Feedforward function produces result of the network prediction for each sample.
    First we find neurons' values for hidden layer, then for output layer.
    
    'the optimal size of the hidden layer is usually between the size of the input and size of the output layers'. 
    Jeff Heaton, the author of Introduction to Neural Networks in Java, offers a few more.

    for most problems, one could probably get decent performance (even without a second optimization step) by setting
    the hidden layer configuration using just two rules: (i) the number of hidden layers equals one;
    and (ii) the number of neurons in that layer is the mean of the neurons in the input and output layers.
    
    (4 + 1)/2 = 2.5, 3 neuron for hidden layer should be enough
    '''
    def feedforward(self, x):
        
        # x[0], x[1] - features
        # n* - neurons in the hidden layer, y_hat - predicted value
        self.n1    = self.sigmoid(x[0]*self.w1 + x[1]*self.w2 + + x[2]*self.w3 + x[3]*self.w4 + self.b_n1) 
        self.n2    = self.sigmoid(x[0]*self.w5 + x[1]*self.w6 + + x[2]*self.w7 + x[3]*self.w8 + self.b_n2) 
        self.n3    = self.sigmoid(x[0]*self.w9 + x[1]*self.w10 + + x[2]*self.w11 + x[3]*self.w12 + self.b_n3)
        self.n4    = self.sigmoid(x[0]*self.w13 + x[1]*self.w14 + + x[2]*self.w15 + x[3]*self.w16 + self.b_n4) 
    
        self.y_hat = self.sigmoid(self.n1*self.w17 + self.n2*self.w18 + self.n3*self.w19 + self.n4*self.w20 + self.b_y_hat)

    
    '''
    Backpropagation updates all the weights and biases of the network.
    By using Gradient Descent technique, each trainable parameter (weight or bias)
    is changing a little bit towards minimum of MSE.
    Unlike forward propagation, we tweak our parameters starting
    from the right end of the network, meaning first we update
    weights and biases for output layer, then for the hidden.
    If we had more than one hidden layer, we would go over them 
    is similar fashion, 
    like this: "output layer" => "hidden layer 2" => "hidden layer 1"
    '''
    def backpropagation(self, x, y):

        # We calculate some values here to use them later
        y_hat_der = (-2 * (y-self.y_hat) * self.sigmoid_der(self.n1*self.w13 + self.n2*self.w14 + self.n3*self.w15  + self.b_y_hat))
        n1_der = self.w17 * self.sigmoid_der(x[0]*self.w1 + x[1]*self.w2 + + x[2]*self.w3 + x[3]*self.w4 + self.b_n1)
        n2_der = self.w18 * self.sigmoid_der(x[0]*self.w5 + x[1]*self.w6 + + x[2]*self.w7 + x[3]*self.w8 + self.b_n2)
        n3_der = self.w19 * self.sigmoid_der(x[0]*self.w9 + x[1]*self.w10 + + x[2]*self.w11 + x[3]*self.w12 + self.b_n3)
        n4_der = self.w20 * self.sigmoid_der(x[0]*self.w13 + x[1]*self.w14 + + x[2]*self.w15 + x[3]*self.w16 + self.b_n4)
        
        # Biases
        self.b_n1    -= self.lr * y_hat_der * n1_der
        self.b_n2    -= self.lr * y_hat_der * n2_der
        self.b_n3    -= self.lr * y_hat_der * n3_der
        self.b_n4    -= self.lr * y_hat_der * n4_der
        self.b_y_hat -= self.lr * y_hat_der

        # Weights
        self.w17 -= self.lr * y_hat_der * self.n1
        self.w18 -= self.lr * y_hat_der * self.n2
        self.w19 -= self.lr * y_hat_der * self.n3
        self.w20 -= self.lr * y_hat_der * self.n4
        
        self.w1 -= self.lr * y_hat_der * n1_der * x[0]
        self.w2 -= self.lr * y_hat_der * n1_der * x[1]
        self.w3 -= self.lr * y_hat_der * n1_der * x[2]
        self.w4 -= self.lr * y_hat_der * n1_der * x[3]
        self.w5 -= self.lr * y_hat_der * n2_der * x[0]
        self.w6 -= self.lr * y_hat_der * n2_der * x[1]
        self.w7 -= self.lr * y_hat_der * n2_der * x[2]
        self.w8 -= self.lr * y_hat_der * n2_der * x[3]
        self.w9 -= self.lr * y_hat_der * n3_der * x[0]
        self.w10 -= self.lr * y_hat_der * n3_der * x[1]
        self.w11 -= self.lr * y_hat_der * n3_der * x[2]
        self.w12 -= self.lr * y_hat_der * n3_der * x[3]
        self.w13 -= self.lr * y_hat_der * n4_der * x[0]
        self.w14 -= self.lr * y_hat_der * n4_der * x[1]
        self.w15 -= self.lr * y_hat_der * n4_der * x[2]
        self.w16 -= self.lr * y_hat_der * n4_der * x[3]
        
    
    '''
    Training process is the combination of forward and back propagations.
    '''
    def fit(self, X, y, epoch=10, lr=0.01):

        mse_list = []
        self.lr = lr

        # Loop to go over epochs. Each epoch train network on all available data.
        # We also check MSE and store it to visualize training process
        for i in range(epoch):
            mse = mean_squared_error(y, self.predict(X))
            mse_list.append(mse)
            print(f'Epoch: {i+1} / {epoch}, MSE: {round(mse, 4)}', end='\r')

            # Loop to go over each training example for current epoch
            for j in range(len(X)):
                self.feedforward(X[j])
                self.backpropagation(X[j], y[j]) 

        return mse_list

    '''
    This function is very similar to feed forward,
    it's in fact uses 'feedforward' function to make predictions.
    The only difference is that we predict outcome for all samples.
    '''
    def predict(self, X):

        result = []

        for x in X:
            self.feedforward(x)
            result.append(self.y_hat)

        return result

R-square : R2=1−MSE(model)MSE(baseline)=1−∑Ni=1(y1−y1^)2∑Ni=1(y1¯−y1^)2
adjusted R-square: R2¯=1−(1−R2)(n−1n−k+1)
                    k: number of features
                    n: number of samples

In [ ]:
model = SimpleNeuralNetwork()
lr_list = [0.001,0.005, 0.01, 0.05]

plt.figure(figsize=(14,7))
plt.style.use('seaborn-whitegrid')
plt.title('Neural Network Training Process', fontsize=15)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('MSE', fontsize=12)

for lr in lr_list:
    history = model.fit(x_train, y_train, epoch=100, lr=lr)
    plt.plot(history,label=f"Learning rate: {lr}")
    
plt.legend()
plt.show()
   

In [ ]:

# keras_model.fit(x_train, y_train, epochs=100, verbose=0)
history = model.fit(x_train, y_train, epoch=100, lr=0.05)
val_pred = model.predict(x_val)
train_pred = model.predict(x_train)
test_pred = model.predict(x_test)

val_acc_score =r2_score(y_val, val_pred)
train_acc_score =r2_score(y_train, train_pred)
test_acc_score =r2_score(y_test, test_pred)

# TODO build other neurons for other 12 features

print(f"Validation Accuracy Score:\t {val_acc_score, r_squared(y_val,val_pred)} ")
print(f"Training Accuracy Score:\t {train_acc_score, r_squared(y_train, train_pred)} ")
print(f"Test Accuracy Score:\t\t {test_acc_score, r_squared(y_test, test_pred)} ")

plt.figure(figsize=(14,7))
plt.scatter(x=y_val,y=val_pred,alpha=0.5)
plt.xlabel('y_test',size=12)
plt.ylabel('y_pred',size=12)
plt.title('Predicited Values vs Original Values (Test Set)',size=15)
plt.show()